<a href="https://colab.research.google.com/github/tiagopecurto/tiagopecurto/blob/main/70705-Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistemas para Processamento de Big Data
## TP1 - Energy Meter Monitoring




The sensor data corresponds to regular readings from 11 residential energy meters. The data covers the month of February 2024.

Each data sample has the following schema:

timestamp | sensor_id | energy
----------|-------------|-----------
timestamp | string  | float

Each energy value (KWh) corresponds to the accumulated value of the meter at the time of measurement. As such,
each meter is expected to produce a monotonically increasing series of pairs of timestamp and energy consummed up to that moment.

The meters do not start at zero or at the same value.


## Questions

The following questions should be answered for the month of February and only for this month.

### For the group of sensors:

1. Compute the total energy consumed.

2. Compute the running total energy consumed so far for each day, inclusive.

Note: You can approximate the result but using the last reading of each day from each sensor.

### For each sensor, separately:

3. Compute the total energy consumed and the average energy consumption per day.

4. Compute the day of the month with minimum and maximum energy consumption.

Note: You can approximate the result but using the last reading of each day from each sensor.

### For each sensor, separately, with estimations:

**Assumptions:**

+ Readings may be missing for extended periods due to communication problems with the sensors.

+ Readings are collected do not fall precisely "on the hour". The are collected and recorded any time.

+ For more precise results, estimate the value of the meter at precise timestamp, using linear interpolation from nearest readings.

5. Compute the **estimated** value of each sensor meter for every hour and day of the month (in ascending order).

6. Compute the **estimated** running total of the energy consumed so far. The value should be updated every hour.

## Requeriments

Solve each question using Structured Spark, either Dataframes or SQL or both.

## Other Grading Criteria

+ Grading will also take into account the general clarity of the programming and of the presentation report (notebook).




### Deadline
+ November 10, 23h59

For each day late, ***0.5 / day penalty***. Penalty accumulates until the grade of the assignment reaches 8.0.

---
### Colab Setup


In [ ]:
#@title Install PySpark
!pip install pyspark findspark --quiet

In [5]:
#@title Download the dataset

!wget -q -O energy-readings.csv https://raw.githubusercontent.com/smduarte/spbd-2425/refs/heads/main/docs/labs/projs/energy-readings.csv
!head -2 energy-readings.csv

date;sensor;energy
2024-02-01 00:00:00;D;2615.0


In [6]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]') \
						.appName('energy').getOrCreate()

sc = spark.sparkContext
try :
    readings = spark.read.csv('energy-readings.csv', sep =';', header=True, inferSchema=True)

    #0.Filtra os dados para manter apenas o mês de fevereiro de 2024
    readings_february = readings.filter((month("date") == 2) & (year("date") == 2024))

except Exception as err:
    print(err)

**Question 2** - For the group of sensors, compute the running total energy consumed so far for each day, inclusive

In [8]:
#2.1: Calcula o consumo diário de cada sensor
daily_consumption_per_sensor = readings_february.withColumn("day", to_date("date")) \
  .groupBy("day", "sensor") \
  .agg(round(max("energy") - min("energy"),3).alias("daily_consumption")) \
  .orderBy("day", "sensor")
#daily_consumption_per_sensor.show()

#2.2: Soma o consumo diário para todos os sensores por dia
total_daily_consumption = daily_consumption_per_sensor.groupBy("day") \
  .agg(round(sum("daily_consumption"),3).alias("Total Daily Consumption")) \
  .orderBy("day")
total_daily_consumption.show(30)

+----------+-----------------------+
|       day|total_daily_consumption|
+----------+-----------------------+
|2024-02-01|                  119.7|
|2024-02-02|                   69.6|
|2024-02-09|                   55.7|
|2024-02-10|                  113.9|
|2024-02-11|                  117.7|
|2024-02-12|                  110.5|
|2024-02-13|                  112.5|
|2024-02-14|                   92.9|
|2024-02-15|                   81.0|
|2024-02-16|                   47.4|
|2024-02-18|                   57.8|
|2024-02-19|                   79.5|
|2024-02-20|                   84.0|
|2024-02-21|                   83.0|
|2024-02-22|                   76.5|
|2024-02-23|                   84.4|
|2024-02-24|                  63.55|
|2024-02-25|                  99.78|
|2024-02-26|                  92.47|
|2024-02-27|                  92.92|
|2024-02-28|                 113.42|
|2024-02-29|                 132.36|
+----------+-----------------------+

